In [25]:
import io
import math
import numpy as np
import os

#Read Zip File
import zipfile

#Read Images and XMLs
from PIL import Image, ImageFilter
import xml.etree.ElementTree as ET

#Visualize Results
import matplotlib.pyplot as plt
%matplotlib inline

# Load XML File


In [18]:
def xml_to_dict(el):
    """
    Concerts XML to Dict. Note that the "object" tag is assigned to key "objects",
    which contains a list of all "object" tags. The "objects" element is placed on 
    the same level as annotation
    """
    my_dict = {}
    objects = []
  
    if(len(list(el))>0):
        my_dict_tmp = {}
        for el_tmp in el:
        
            if el_tmp.tag == "object":
                objects.append(xml_to_dict(el_tmp))
            else:
                my_dict_tmp.update(xml_to_dict(el_tmp))
    
        if len(objects) > 0:
            my_dict['objects'] = objects
    
        my_dict[el.tag] = my_dict_tmp
    else:
        my_dict[el.tag] = el.text
  
    return my_dict

In [23]:
import glob


test_xml = {}

path = r'C:\\Users\\c10670A\\Documents\\Python Scripts\\03. Projects\\Kaufland_Case\\Kaufland_DataThon_2019_testset_xml\\test_xml_edited\\*.xml'
files = glob.glob(path)
for name in files:
    with open(name) as f:
        test_xml[name.split("\\")[-1][:-4]] = xml_to_dict(ET.fromstring(f.read().replace('\n', '')))
    

# Load Output

In [67]:
pred_data = r"C:\Users\c10670A\Documents\Python Scripts\03. Projects\Kaufland_Case"
test_pred = os.path.join(pred_data, "output-test.txt")


f = open(test_pred, "r")
test_map_obj = {}
for l in f:
    l = l.replace("\n", "")
    data = l.split(",")
    imgName = data[0].split("/")[-1][:-4].replace("_small", "")    
    test_map_obj[imgName] = []

f.close()

f = open(test_pred, "r")
for l in f:
    l = l.replace("\n", "")
    data = l.split(",")
    imgName = data[0].split("/")[-1][:-4].replace("_small", "")
    
    test_map_obj[imgName].append({'type' : int(data[1]), 'position' : [int(int(x) * 11.52) for x in data[3:]]})

f.close()

In [68]:
test_map_obj

{'IMG_20190306_094644': [{'type': 0, 'position': [3421, 2534, 4216, 2926]},
  {'type': 0, 'position': [610, 2534, 1509, 2903]},
  {'type': 0, 'position': [1935, 2522, 2638, 2903]},
  {'type': 1, 'position': [714, 1693, 1670, 2373]},
  {'type': 1, 'position': [2684, 1313, 3294, 2396]},
  {'type': 1, 'position': [3479, 1117, 4055, 2453]},
  {'type': 1, 'position': [2960, 979, 3386, 2626]},
  {'type': 1, 'position': [4078, 1071, 4619, 2592]},
  {'type': 1, 'position': [1267, 1497, 1912, 2315]},
  {'type': 1, 'position': [1681, 1474, 2292, 2280]}],
 'IMG_20190306_094725': [{'type': 0, 'position': [3168, 2741, 4020, 3156]},
  {'type': 0, 'position': [599, 2718, 1578, 3121]},
  {'type': 0, 'position': [1762, 2718, 2753, 3144]},
  {'type': 0, 'position': [4412, 2776, 4561, 3121]},
  {'type': 1, 'position': [3075, 1013, 3559, 2718]},
  {'type': 1, 'position': [3548, 1094, 4008, 2684]},
  {'type': 1, 'position': [1774, 1785, 2315, 2638]},
  {'type': 1, 'position': [864, 1843, 1566, 2580]},
  {'

# Assign final flag

In [69]:
def check_intersection(first, second):
    return not (first[2] < second[0] or \
                    first[0] >  second[2] or \
                    first[3] < second[1] or \
                    first[1] > second[3])

In [76]:
def are_of_intersection(first, second):
    si = max(0, min(first[2], second[2]) - max(first[0], second[0])) * max(0, min(first[3], second[3]) - max(first[1], second[1]))
    su = (first[3]-first[1])*(first[2]-first[0]) + (second[3]-second[1])*(second[2]-second[0]) - si
    return si/su
    
    

In [87]:
list_files = ["IMG_20190306_094945","IMG_20190306_094644","IMG_20190306_094725","IMG_20190306_094834",
"IMG_20190306_094849","IMG_20190306_094904","IMG_20190306_094931","IMG_20190306_094812",
"IMG_20190306_094948","IMG_20190306_094730","IMG_20190306_094910","IMG_20190306_094937",
"IMG_20190306_094923","IMG_20190306_094907","IMG_20190306_094855","IMG_20190306_094926",
"IMG_20190306_094915","IMG_20190306_094819","IMG_20190306_094941","IMG_20190306_094808"]


In [102]:
result = []
for key in list_files:
    value = test_xml[key]
    for obj in value['objects']:
        name = obj['object']['name']
        
        if 'label' in name:
            flag = 1
            
            for pred_obj in test_map_obj[key]:
                if pred_obj['type'] == 0:
                    if check_intersection([int(x) for x in list(obj['object']['bndbox'].values())],
                                    pred_obj['position']):
                        flag = 0
                        break
            
        else:
            flag = 1
            
            for pred_obj in test_map_obj[key]:
                if pred_obj['type'] == 1:
                    if check_intersection([int(x) for x in list(obj['object']['bndbox'].values())],
                                    pred_obj['position']):
                        overl = are_of_intersection([int(x) for x in list(obj['object']['bndbox'].values())],
                                    pred_obj['position'])
                        
                        if overl > 0.20:
                            flag = 0
                            break
        
        result.append(name + "," + key +","+ str(flag))
    

In [103]:
for i in result:
    print(i)

00015907,IMG_20190306_094945,1
label_00015907,IMG_20190306_094945,0
label_02802511,IMG_20190306_094945,0
02802511,IMG_20190306_094945,1
label_03001210,IMG_20190306_094644,0
03001210,IMG_20190306_094644,0
03001210,IMG_20190306_094644,1
label_03001211,IMG_20190306_094644,0
03001211,IMG_20190306_094644,1
label_03001220,IMG_20190306_094725,0
03001220,IMG_20190306_094725,0
03001220,IMG_20190306_094725,0
03001220,IMG_20190306_094725,0
03001220,IMG_20190306_094725,0
label_03100931,IMG_20190306_094725,0
03100931,IMG_20190306_094725,0
03100931,IMG_20190306_094725,0
label_00014557,IMG_20190306_094725,0
00014557,IMG_20190306_094725,0
00014557,IMG_20190306_094725,0
label_00011325,IMG_20190306_094834,0
label_00054501,IMG_20190306_094834,0
label_00062447,IMG_20190306_094834,0
00062436,IMG_20190306_094834,0
00062436,IMG_20190306_094834,0
label_00062435,IMG_20190306_094834,0
00062435,IMG_20190306_094834,1
label_00062538,IMG_20190306_094849,0
label_00062541,IMG_20190306_094849,0
label_00011325,IMG_2019

In [57]:
test_xml

{'IMG_20190306_094644': {'objects': [{'object': {'name': 'label_03001210',
     'pose': 'Unspecified',
     'truncated': '0',
     'difficult': '0',
     'bndbox': {'xmin': '636',
      'ymin': '2532',
      'xmax': '1473',
      'ymax': '2957'}}},
   {'object': {'name': '03001210',
     'pose': 'Unspecified',
     'truncated': '0',
     'difficult': '0',
     'bndbox': {'xmin': '942',
      'ymin': '1639',
      'xmax': '1558',
      'ymax': '2471'}}},
   {'object': {'name': '03001210',
     'pose': 'Unspecified',
     'truncated': '0',
     'difficult': '0',
     'bndbox': {'xmin': '1563',
      'ymin': '1650',
      'xmax': '2037',
      'ymax': '2332'}}},
   {'object': {'name': 'label_03001211',
     'pose': 'Unspecified',
     'truncated': '0',
     'difficult': '0',
     'bndbox': {'xmin': '1903',
      'ymin': '2532',
      'xmax': '2693',
      'ymax': '2956'}}},
   {'object': {'name': '03001211',
     'pose': 'Unspecified',
     'truncated': '0',
     'difficult': '0',
     'b

In [ ]:
max()